In [1]:
# Load libraries
import datetime
import numpy as np
from collections import Counter
import torch
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os.path



import mne
import braindecode
from torch.utils.data import DataLoader

from braindecode.preprocessing import (
    preprocess, Preprocessor, create_fixed_length_windows, scale as multiply)
from braindecode.datautil import load_concat_dataset
from braindecode.datasets import BaseConcatDataset

import skorch
from skorch.helper import SliceDataset

from braindecode.models import ShallowFBCSPNet, Deep4Net, EEGResNet, EEGNetv4, TCN, HybridNet
import torch 
from torch.optim import swa_utils


from skorch.callbacks import LRScheduler
from skorch.helper import predefined_split
from sklearn.metrics import accuracy_score,balanced_accuracy_score


from braindecode import EEGClassifier

from sklearn.model_selection import GridSearchCV

mne.set_log_level('ERROR')  # avoid messages everytime a window is extracted

from IPython.utils import io
from braindecode.datautil.windowers import create_fixed_length_windows



/home/mila/m/mohammad-javad.darvishi-bayasi/.conda/envs/braindecode/lib/python3.8/site-packages/braindecode/datautil/windowers.py:4: UserWarning: datautil.windowers module is deprecated and is now under preprocessing.windowers, please use from import braindecode.preprocessing.windowers
  warn('datautil.windowers module is deprecated and is now under '


In [2]:
TUH_PATH = '/home/mila/m/mohammad-javad.darvishi-bayasi/scratch/medical/eeg/tuab/tuab_org/'
TUH_PATH_pp = '/home/mila/m/mohammad-javad.darvishi-bayasi/scratch/medical/eeg/tuab/tuab_pp3'

NMT_PATH = '/home/mila/m/mohammad-javad.darvishi-bayasi/scratch/medical/eeg/NMT/nmt_scalp_eeg_dataset/'
NMT_PATH_pp = '/home/mila/m/mohammad-javad.darvishi-bayasi/scratch/medical/eeg/NMT/nmt_pp3'

N_JOBS = 8  # specify the number of jobs for loading and windowing

Analyze similar subjects in train vs test

In [3]:
dataset = 'tuh'

if (dataset == 'tuh') or (dataset == 'merge'):
    tuh_preproc = load_concat_dataset(
    path=TUH_PATH_pp,
    preload=False,
    # ids_to_load=range(50), #or None to load the whole dataset,
    target_name=( 'pathological'),#,'age','gender'),  # 'age', 'gender',
    )
    tuh_preproc.description
    selected_ds = tuh_preproc

if (dataset == 'nmt') or (dataset == 'merge'):
    nmt_preproc = load_concat_dataset(
    path=NMT_PATH_pp,
    preload=False,
    # ids_to_load=range(100), #range(1000), #or None to load the whole dataset,
    target_name=( 'pathological'),#,'age','gender'),  # 'age', 'gender',
    )
    nmt_preproc.set_description({
    "subject": [ii for ii in range(len(nmt_preproc.description))]})

    # nmt_preproc = shuffle_train_coulmn(nmt_preproc)
    selected_ds = nmt_preproc

if dataset == 'merge':
    selected_ds = BaseConcatDataset([nmt_preproc,tuh_preproc])

In [4]:
df = selected_ds.description
df[:5]

,year,month,day,path,subject,session,segment,age,gender,version,train,pathological
0,1899,12,30,/home/mila/m/mohammad-javad.darvishi-bayasi/sc...,aaaaadsy,3,1,56,M,eeg,True,True
1,1899,12,30,/home/mila/m/mohammad-javad.darvishi-bayasi/sc...,aaaaajga,1,1,23,M,eeg,False,False
2,1899,12,30,/home/mila/m/mohammad-javad.darvishi-bayasi/sc...,aaaaaker,2,1,51,F,eeg,True,True
3,1899,12,30,/home/mila/m/mohammad-javad.darvishi-bayasi/sc...,aaaaakjw,3,1,64,M,eeg,True,True
4,1899,12,30,/home/mila/m/mohammad-javad.darvishi-bayasi/sc...,aaaaakmw,9,1,51,M,eeg,True,True


In [5]:
# Split the dataframe by pathological column
groups = df.groupby('pathological')

# Access each group as a dataframe
df_yes = groups.get_group(True)
df_no = groups.get_group(False)

# Display the first five rows of each group
df_yes.head()
df_no.head()

# Get the unique subjects in each group
subjects_yes = set(df_yes['subject'].unique())
subjects_no = set(df_no['subject'].unique())

# Find the intersection of the two sets
common_subjects = subjects_yes.intersection(subjects_no)

# Display the common subjects
print(f'there are {len(common_subjects)} common subjects in the two pathological groups')

# Filter the dataframe by the common subjects
df_common = df[df['subject'].isin(common_subjects)]

# Display the number of rows of the filtered dataframe
print(f'There are {len(df_common)} out of {len(df)} rows have their status changed')

# Create a new column named 'common' with boolean values
df = df.assign(common=df['subject'].isin(common_subjects))

# Display the first five rows of the dataframe
df.head()

there are 54 common subjects in the two pathological groups
There are 149 out of 2993 rows have their status changed


,year,month,day,path,subject,session,segment,age,gender,version,train,pathological,common
0,1899,12,30,/home/mila/m/mohammad-javad.darvishi-bayasi/sc...,aaaaadsy,3,1,56,M,eeg,True,True,False
1,1899,12,30,/home/mila/m/mohammad-javad.darvishi-bayasi/sc...,aaaaajga,1,1,23,M,eeg,False,False,False
2,1899,12,30,/home/mila/m/mohammad-javad.darvishi-bayasi/sc...,aaaaaker,2,1,51,F,eeg,True,True,False
3,1899,12,30,/home/mila/m/mohammad-javad.darvishi-bayasi/sc...,aaaaakjw,3,1,64,M,eeg,True,True,False
4,1899,12,30,/home/mila/m/mohammad-javad.darvishi-bayasi/sc...,aaaaakmw,9,1,51,M,eeg,True,True,False


In [6]:
df_common.sort_values(by='subject', ascending=False).head(20)

,year,month,day,path,subject,session,segment,age,gender,version,train,pathological
2875,2013,8,12,/home/mila/m/mohammad-javad.darvishi-bayasi/sc...,aaaaappt,2,0,33,M,eeg,True,False
2829,2013,7,29,/home/mila/m/mohammad-javad.darvishi-bayasi/sc...,aaaaappt,1,1,33,M,eeg,True,True
2766,2013,7,9,/home/mila/m/mohammad-javad.darvishi-bayasi/sc...,aaaaapnl,2,1,55,M,eeg,True,True
2771,2013,7,10,/home/mila/m/mohammad-javad.darvishi-bayasi/sc...,aaaaapnl,3,0,55,M,eeg,True,True
2753,2013,7,4,/home/mila/m/mohammad-javad.darvishi-bayasi/sc...,aaaaapnl,1,1,55,M,eeg,True,False
2817,2013,7,26,/home/mila/m/mohammad-javad.darvishi-bayasi/sc...,aaaaapnc,19,1,49,F,eeg,True,True
2816,2013,7,26,/home/mila/m/mohammad-javad.darvishi-bayasi/sc...,aaaaapnc,18,0,49,F,eeg,True,True
2785,2013,7,15,/home/mila/m/mohammad-javad.darvishi-bayasi/sc...,aaaaapnc,1,0,86,F,eeg,True,False
2704,2013,6,18,/home/mila/m/mohammad-javad.darvishi-bayasi/sc...,aaaaaoru,2,0,44,M,eeg,True,True
2741,2013,7,1,/home/mila/m/mohammad-javad.darvishi-bayasi/sc...,aaaaaoru,3,1,44,M,eeg,True,True


In [7]:
df[df['subject']=='aaaaapnc']

,year,month,day,path,subject,session,segment,age,gender,version,train,pathological,common
2785,2013,7,15,/home/mila/m/mohammad-javad.darvishi-bayasi/sc...,aaaaapnc,1,0,86,F,eeg,True,False,True
2816,2013,7,26,/home/mila/m/mohammad-javad.darvishi-bayasi/sc...,aaaaapnc,18,0,49,F,eeg,True,True,True
2817,2013,7,26,/home/mila/m/mohammad-javad.darvishi-bayasi/sc...,aaaaapnc,19,1,49,F,eeg,True,True,True


In [8]:
# Create a new column named 'common' with boolean values
df = df.assign(common=df['subject'].isin(common_subjects))

# Display the first five rows of the dataframe
# df.head()
selected_ds.set_description(df, overwrite=True)

ds_all = selected_ds.split('common')['False']
ds_all.description

,year,month,day,path,subject,session,segment,age,gender,version,train,pathological,common
0,1899,12,30,/home/mila/m/mohammad-javad.darvishi-bayasi/sc...,aaaaadsy,3,1,56,M,eeg,True,True,False
1,1899,12,30,/home/mila/m/mohammad-javad.darvishi-bayasi/sc...,aaaaajga,1,1,23,M,eeg,False,False,False
2,1899,12,30,/home/mila/m/mohammad-javad.darvishi-bayasi/sc...,aaaaaker,2,1,51,F,eeg,True,True,False
3,1899,12,30,/home/mila/m/mohammad-javad.darvishi-bayasi/sc...,aaaaakjw,3,1,64,M,eeg,True,True,False
4,1899,12,30,/home/mila/m/mohammad-javad.darvishi-bayasi/sc...,aaaaakmw,9,1,51,M,eeg,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2839,2013,10,7,/home/mila/m/mohammad-javad.darvishi-bayasi/sc...,aaaaajah,4,0,40,M,eeg,False,True,False
2840,2013,10,7,/home/mila/m/mohammad-javad.darvishi-bayasi/sc...,aaaaalmj,3,1,40,F,eeg,True,False,False
2841,2013,10,7,/home/mila/m/mohammad-javad.darvishi-bayasi/sc...,aaaaanth,4,1,22,M,eeg,True,True,False
2842,2013,10,10,/home/mila/m/mohammad-javad.darvishi-bayasi/sc...,aaaaaeuq,2,1,55,F,eeg,True,False,False


In [9]:
# Split the dataframe by pathological column
groups = df.groupby('train')

# Access each group as a dataframe
df_yes = groups.get_group(True)
df_no = groups.get_group(False)

# Display the first five rows of each group
df_yes.head()
df_no.head()

# Get the unique subjects in each group
subjects_yes = set(df_yes['subject'].unique())
subjects_no = set(df_no['subject'].unique())

# Find the intersection of the two sets
common_subjects = subjects_yes.intersection(subjects_no)

# Display the common subjects
print(f'there are {len(common_subjects)} common subjects in the train/test')


there are 0 common subjects in the train/test


In [10]:
df = df.assign(gender_bool=df['gender'].map(lambda x: 0 if x == 'M' else 1 if x == 'F' else None))

In [11]:
df['gender_bool']==True

0       False
1       False
2        True
3       False
4       False
        ...  
2988    False
2989     True
2990    False
2991     True
2992     True
Name: gender_bool, Length: 2993, dtype: bool

confounders?!

In [13]:
## data windowing
train_set = selected_ds.split('train')['True']
test_set = selected_ds.split('train')['False']

input_window_samples =6000
n_preds_per_input = 5542

with io.capture_output() as captured:
        window_train_set = create_fixed_length_windows(train_set, 
                                                    start_offset_samples=0,
                                                    stop_offset_samples=None,
                                                    preload=True,
                                                    window_size_samples=input_window_samples,
                                                    window_stride_samples=n_preds_per_input,
                                                    drop_last_window=True,
                                                    # mapping={ False: 0, True: 1 , 'M': 0, 'F': 1 },  # map non-digit targets 'M': 0, 'F': 1,
                                                    )
# with io.capture_output() as captured:
#         window_val_set = create_fixed_length_windows(val_set, 
#                                                     start_offset_samples=0,
#                                                     stop_offset_samples=None,
#                                                     preload=True,
#                                                     window_size_samples=input_window_samples,
#                                                     window_stride_samples=n_preds_per_input,
#                                                     drop_last_window=True,)
with io.capture_output() as captured:
        window_test_set = create_fixed_length_windows(test_set,
                                                    start_offset_samples=0,
                                                    stop_offset_samples=None,preload=False,
                                                    window_size_samples=input_window_samples,
                                                    window_stride_samples=n_preds_per_input,
                                                    drop_last_window=False,)


In [14]:
# Model and classifier
n_chans =21
n_classes = 2
n_start_chans = 25
final_conv_length = 1
n_chan_factor = 2
stride_before_pool = True
drop_prob = 0.0
# input_window_samples =6000
model = Deep4Net(
            n_chans, n_classes,
            n_filters_time=n_start_chans,
            n_filters_spat=n_start_chans,
            input_window_samples=input_window_samples,
            n_filters_2=int(n_start_chans * n_chan_factor),
            n_filters_3=int(n_start_chans * (n_chan_factor ** 2.0)),
            n_filters_4=int(n_start_chans * (n_chan_factor ** 3.0)),
            final_conv_length=final_conv_length,
            stride_before_pool=stride_before_pool,
            drop_prob=drop_prob)

In [16]:
from braindecode import EEGClassifier
from braindecode.training.losses import CroppedLoss

batch_size = 64
lr = 0.001

clf = EEGClassifier(
                model,
                cropped=True,
                iterator_train=DataLoader,  # This tells EEGClassifier to use a custom DataLoader
                # iterator_train=AugmentedDataLoader,  # This tells EEGClassifier to use a custom DataLoader
                # iterator_train__transforms=transforms_train,  # This sets the augmentations to use
                iterator_valid =DataLoader,  # This tells EEGClassifier to use a custom DataLoader
                # iterator_valid =AugmentedDataLoader,  # This tells EEGClassifier to use a custom DataLoader
                # iterator_valid__transforms=transforms_val,  # This sets the augmentations to use
                criterion=CroppedLoss,
                # criterion=CroppedLoss_sd,
                criterion__loss_function=torch.nn.functional.nll_loss,
                optimizer=torch.optim.AdamW,
                # optimizer__param_groups=optimizer__param_groups,
                # optimizer=dis_lr,
                train_split=predefined_split(window_train_set),
                # train_split=predefined_split(window_test_set),
                optimizer__lr=lr,
                # optimizer__weight_decay=weight_decay,
                iterator_train__shuffle=False,
                # iterator_train__sampler = ImbalancedDatasetSampler(window_train_set, labels=window_train_set.get_metadata().target),
                # iterator_train__sampler = ImbalancedDatasetSampler_with_ds(window_train_set, labels=window_train_set.get_metadata().target, dataset_label=window_train_set.get_metadata().dataset),
                # iterator_valid__sampler = ImbalancedDatasetSampler_with_ds(window_val_set, labels=window_val_set.get_metadata().target, dataset_label=window_val_set.get_metadata().dataset),
                batch_size=batch_size,
                callbacks=[
                    # EarlyStopping(patience=5),
                    # StochasticWeightAveraging(swa_utils, swa_start=1, verbose=1, swa_lr=lr),
                    "accuracy", "balanced_accuracy","f1",
                    # ("lr_scheduler", LRScheduler('CosineAnnealingLR', T_max=n_epochs - 1)),
                    # WandbLogger(wandb_run),
                    # checkpoint,
                    # freezer,
                    ],  #"accuracy",
                device='cuda')



In [21]:
clf.initialize() # This is important!
load_path = '/home/mila/m/mohammad-javad.darvishi-bayasi/scratch/medical/eeg/results_pp3/Deep4Net/seed100/TUH_deep4_hps_pp3_wN_wAug_WoPt81_index9_number2700/Deep4Net_trained_TUH_deep4_hps_pp3_wN_wAug_WoPt81_index9_number2700_state_dict_100.pt'
state_dicts = torch.load(load_path)# +'state_dict_2023.pt')
model.load_state_dict(state_dicts, strict= False)
print('pre-trained model loaded using pytorch')

Re-initializing module.
Re-initializing criterion because the following parameters were re-set: loss_function.
Re-initializing optimizer.
pre-trained model loaded using pytorch


In [47]:
ds=window_train_set

In [48]:
test_X = SliceDataset(ds, idx=0)
test_y = ds.get_metadata().target #.to_numpy()
# print(test_y)
# test_y = test_set.get_metadata().gender_bool #.to_numpy()
# print(test_y)
y_true =test_y#[:,0]
b_acc = balanced_accuracy_score(np.array(y_true), clf.predict(test_X))
# loss_merge = loss_scoring (clf,test_X,test_y) 
# print('loss_merge:', loss_merge)
print('B_acc:', b_acc)

B_acc: 0.8173445809665941


In [49]:
df = ds.get_metadata()
df = df.assign(y_pred=clf.predict(test_X))
# df

In [50]:
balanced_accuracy_score(np.array(df['target']), np.array(df['y_pred']))

0.8173445809665941

In [51]:
# Split the dataframe by the gender column
groups = df.groupby('gender')

# Access each sub-dataframe as a group
df_m = groups.get_group('M')
df_f = groups.get_group('F')

groups = df_m.groupby('pathological')
df_mn = groups.get_group(False)
df_ma = groups.get_group(True)

groups = df_f.groupby('pathological')
df_fn = groups.get_group(False)
df_fa = groups.get_group(True)


In [52]:
accuracy_score(np.array(df_m['target']), np.array(df_m['y_pred']))

0.8143307086614173

In [53]:
accuracy_score(np.array(df_f['target']), np.array(df_f['y_pred']))

0.8221147201105736

In [54]:
print(accuracy_score(np.array(df_mn['target']), np.array(df_mn['y_pred'])))
print(accuracy_score(np.array(df_ma['target']), np.array(df_ma['y_pred'])))
print(accuracy_score(np.array(df_fn['target']), np.array(df_fn['y_pred'])))
print(accuracy_score(np.array(df_fa['target']), np.array(df_fa['y_pred'])))

0.9296849087893864
0.7100449775112444
0.9486979166666667
0.678939617083947
